
# Setup

In [ ]:
import os
import urllib.request
import zipfile
import shutil
from pathlib import Path

from google.colab import drive
drive.mount('/content/drive')

# use until fix is merged into Pytube main
!git clone https://github.com/oncename/pytube.git
!pip install /content/pytube/

os.chdir("/content/drive/MyDrive/ColabFiles/yolo/")

# !pip3 install pytube
!git clone https://github.com/ultralytics/yolov5
!pip install -r yolov5/requirements.txt
!pip3 install torch torchvision
!pip3 install wheel==0.34.1

Mounted at /content/drive
Cloning into 'pytube'...
remote: Enumerating objects: 6996, done.
remote: Counting objects: 100% (715/715), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 6996 (delta 645), reused 630 (delta 598), pack-reused 6281
Receiving objects: 100% (6996/6996), 19.00 MiB | 39.54 MiB/s, done.
Resolving deltas: 100% (4660/4660), done.
Processing ./pytube
  Preparing metadata (setup.py) ... done
  Created wheel for pytube: filename=pytube-15.0.0-py3-none-any.whl size=57565 sha256=299d2bbd75771ea67c34344d27a2d82695cfca527d6b65e41c32fb5db6611c4a
  Stored in directory: /tmp/pip-ephem-wheel-cache-eu6a2y_l/wheels/11/6f/c9/9d00793d27d290628c88821d855482e01d92e375d177ca1259
Successfully built pytube
fatal: destination path 'yolov5' already exists and is not an empty directory.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.6/605.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
os.getcwd()

'/content/drive/MyDrive/ColabFiles/yolo'

In [ ]:
import torch

#'mps' if training via Apple silicon
accelerator = 'cu80'

print('Torch', torch.__version__, 'CUDA', torch.version.cuda)
print('Device:', torch.device('cuda:0'))
print(accelerator)
print(torch.cuda.is_available())

Torch 2.0.1+cu118 CUDA 11.8
Device: cuda:0
cu80
True


# YouTube Download




In [ ]:
from pytube import YouTube
import re

In [ ]:
url_list = [
    'https://www.youtube.com/watch?v=jWCFoQvyKWg&t=928s',
]

file_list = []

for url in url_list:
    yt_video = YouTube(url, use_oauth=True)
    yt_title = yt_video.title
    print(yt_title)
    yt_length = yt_video.length
    print(str(yt_length) + 's')

    pattern = re.compile('[\W_]+')
    file_name = pattern.sub('', yt_title)
    yt_video = yt_video.streams.get_highest_resolution()
    yt_video.download(output_path='./videos/mp4/', filename=file_name + '.mp4', timeout = 5)
    file_list.append(file_name)
    print('your video is downloaded successfully')

LOUD vs. EG — VALORANT Masters Tokyo — Playoffs
5093s
your video is downloaded successfully


# Frame Extraction

In [ ]:
import cv2
from cv2 import error

def extractImages(pathIn, pathOut):
    count = 0
    vidcap = cv2.VideoCapture(pathIn)
    success,image = vidcap.read()
    success = True
    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))
        success,image = vidcap.read()
        print ('Read a new frame: ', success)
        cv2.imwrite(pathOut + "\\frame%d.jpg" % count, image)
        count = count + 1

In [ ]:
for file_name in file_list:
    if ~os.path.exists("./videos/frames/" + file_name):
        os.mkdir("./videos/frames/" + file_name)

    file_location = './videos/mp4/' + file_name + ".mp4"
    print(file_location)

    count = 0
    vidcap = cv2.VideoCapture(file_location)
    success, image = vidcap.read()
    success = True

    while success:
        try:
            vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))    # added this line
            success,image = vidcap.read()
            if count % 100 == 0:
                print('Read frame number ' + str(count)  + ': ', success)
            cv2.imwrite( "./videos/frames/" + file_name + "/frame%d.jpg" % count, image)    # save frame as JPEG file
            count = count + 1
        except error:
            print("done extracting frames")

./videos/mp4/LOUDvsEGVALORANTMastersTokyoPlayoffs.mp4
Read frame number 0:  True
Read frame number 100:  True
Read frame number 200:  True
Read frame number 300:  True
Read frame number 400:  True
Read frame number 500:  True
Read frame number 600:  True
Read frame number 700:  True
Read frame number 800:  True
Read frame number 900:  True
Read frame number 1000:  True
Read frame number 1100:  True
Read frame number 1200:  True
Read frame number 1300:  True
Read frame number 1400:  True
Read frame number 1500:  True
Read frame number 1600:  True
Read frame number 1700:  True
Read frame number 1800:  True
Read frame number 1900:  True
Read frame number 2000:  True
Read frame number 2100:  True
Read frame number 2200:  True
Read frame number 2300:  True
Read frame number 2400:  True
Read frame number 2500:  True
Read frame number 2600:  True
Read frame number 2700:  True
Read frame number 2800:  True
Read frame number 2900:  True
Read frame number 3000:  True
Read frame number 3100:  Tru

# Run Model Against Frames via CMD/Terminal
(Only one of the following 2 execution cells need to be run. Remove comment if running via CMD)


In [ ]:
# !python yolov5/detect.py --source videos/frames/EGvsC9VCTAmericasStage1W1D2Map1/ --weights yolov5/runs/train/wolves_hpe_detection/weights/best.pt --conf 0.6 --name eg_c9_frame_detection --save-txt

# Run Model Against Frames via Python

In [ ]:
import pandas as pd
import numpy as np
import time

model = torch.hub.load('ultralytics/yolov5', 'custom', './yolov5/runs/train/wolves_hpe_detection20/weights/best.pt')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-7-31 Python-3.10.6 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 267 layers, 46113663 parameters, 0 gradients, 107.7 GFLOPs
Adding AutoShape... 


In [ ]:
for file_name in file_list:
    directory = './videos/frames/' + file_name + '/'

    df = pd.DataFrame(columns=["xmin", "ymin", "xmax", "ymax", "confidence", "class", "name", "filename"])

    t0 = time.time()
    for filename in os.listdir(directory):
        print(filename)
        results = model(directory + filename)
        df_single = results.pandas().xyxy[0]
        df_single['filename'] = filename

        df = pd.concat([df, df_single])
    print(file_name + ' inference done: ' + str(time.time() - t0) + 's')

    df.to_csv('./csv/' + file_name +'.csv')

frame2226.jpg
frame2227.jpg
frame2228.jpg
frame2229.jpg
frame2230.jpg
frame2231.jpg
frame2232.jpg
frame2233.jpg
frame2234.jpg
frame2235.jpg
frame2236.jpg
frame2237.jpg
frame2238.jpg
frame2239.jpg
frame2240.jpg
frame2241.jpg
frame2242.jpg
frame2243.jpg
frame2244.jpg
frame2245.jpg
frame2246.jpg
frame2247.jpg
frame2248.jpg
frame2249.jpg
frame2250.jpg
frame2251.jpg
frame2252.jpg
frame2253.jpg
frame2254.jpg
frame2255.jpg
frame2256.jpg
frame2257.jpg
frame2258.jpg
frame2259.jpg
frame2260.jpg
frame2261.jpg
frame2262.jpg
frame2263.jpg
frame2264.jpg
frame2265.jpg
frame2266.jpg
frame2267.jpg
frame2268.jpg
frame2269.jpg
frame2270.jpg
frame2271.jpg
frame2272.jpg
frame2273.jpg
frame2274.jpg
frame2275.jpg
frame2276.jpg
frame2277.jpg
frame2278.jpg
frame2279.jpg
frame2280.jpg
frame2281.jpg
frame2282.jpg
frame2283.jpg
frame2284.jpg
frame2285.jpg
frame2286.jpg
frame2287.jpg
frame2288.jpg
frame2289.jpg
frame2290.jpg
frame2291.jpg
frame2292.jpg
frame2293.jpg
frame2294.jpg
frame2295.jpg
frame2296.jpg
frame2

# Analysis

In [ ]:
display(df)

In [ ]:
hpe = df[df.name == 'hpe']
hpe_single = hpe.drop_duplicates(subset = ['filename'])

In [ ]:
for cmin in np.linspace(0.5, 0.9, 5):
  print(
      "CI>=" + str(cmin) + ": " +
      str(len(hpe_single[hpe_single.confidence >= cmin]))
  )

In [ ]:
doge = df[df.name == 'wolves']
doge_single = doge.drop_duplicates(subset = ['filename'])

In [ ]:
for cmin in np.linspace(0.5, 0.9, 5):
  print(
      "CI>=" + str(cmin) + ": " +
      str(len(doge_single[doge_single.confidence >= cmin]))
  )